In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd # data processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler #good
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
from sklearn import metrics #accuracy measure
import joblib
import warnings
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn import metrics
import random

In [10]:
def load_data_train(files):
    df=None
    for data in files:
        if data.endswith('.csv'):
            file=f"./{data}"
        if df is None:
            df=pd.read_csv(file)
        else:
            d=pd.read_csv(file)
            df=pd.concat([df,d])
            df['label']=df['label'].astype("category")
    return df
def load_data_test(file):
    df=pd.read_csv(file)
    return df
def return_X_Y(df):
    X=df.drop(["label","previous_label"],axis=1)
    y=df['label']
    return X,y
def HGBC_model(X_train,y_train):
    sm = SMOTE()
    all_feature_smote, all_label_smote = sm.fit_resample(X_train,y_train)
    HGBC=HistGradientBoostingClassifier(random_state=10)
    HGBC.fit(all_feature_smote, all_label_smote )
    
    return HGBC
def result_export(model,X_test,y_test):
    accuracy=None
    f1_weighted=None
    f1_macro=None
    model=model
    pred=model.predict(X_test)
    acc=metrics.accuracy_score(y_test,pred)
    f1_weighted_avg=metrics.f1_score(y_test,pred,average='weighted')
    f1_macro_avg=metrics.f1_score(y_test,pred,average='macro')

    return [acc,f1_weighted_avg,f1_macro_avg]
def random_file(folder, number_of_files=3):
    files=os.listdir(f'./{folder}')
    files=[file.split(".")[0][:3] for file in files]
    files = list(set(files))
    files_random=np.random.choice(files,number_of_files,replace=False)
    
    return_files_T1=[file+"T1.csv" for file in files_random]
    return_files_T2=[file+"T2.csv" for file in files_random]
    return return_files_T1 + return_files_T2
def train_set(folder,test_set):
    files=os.listdir(f"./{folder}")
    files_out=[f'./{folder}/'+e for e in files if e not in test_set]
    return files_out
def output_result():
    accuracys=[]
    f1_weighteds=[]
    f1_macros=[]
    
    nurse_test=random_file('nurse')
    student_test=random_file('student')
    nurse_train=train_set('nurse',nurse_test)
    student_train=train_set('student',student_test)
    nurse_test=['./nurse/'+nurse for nurse in nurse_test]
    student_test=['./student/'+student for student in student_test]
    train=nurse_train+student_train+['./GAN/GAN_win2.csv']
    tests=nurse_test+student_test

#     load df
    df=load_data_train(train)
    X,y=return_X_Y(df)
    print('load done')
    
#     train model
    model=HGBC_model(X,y)
    print('model done')
    
#     test
    for test in tests:
        df_test=load_data_test(test)
        X_test,y_test=return_X_Y(df_test)
        [accuracy1,f1_weighted1,f1_macro1]=result_export(model,X_test,y_test)
        accuracys.append(accuracy1)
        f1_weighteds.append(f1_weighted1)
        f1_macros.append(f1_macro1)
        
    print('test done')
    return [np.mean(accuracys),np.std(accuracys),np.mean(f1_weighteds),
            np.std(f1_weighteds),np.mean(f1_macros),np.std(f1_macros)]

In [11]:
results=[]
for i in range(10):
    result=output_result()
    results.append(result)
data=pd.DataFrame(results,columns=['accuracy mean',
                                   'accuracy std',
                                   'f1-score weighted avg mean',
                                  'f1-score weighted avg mean',
                                   'f1-score macro avg mean',
                                  'f1-score macro avg mean'])    

load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done
load done
model done
test done


In [12]:
data.describe()

,accuracy mean,accuracy std,f1-score weighted avg mean,f1-score weighted avg mean,f1-score macro avg mean,f1-score macro avg mean
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.655365,0.074358,0.627165,0.081133,0.496932,0.130843
std,0.022712,0.011940,0.024932,0.010304,0.032211,0.012004
min,0.614467,0.053923,0.587752,0.063597,0.452532,0.111891
25%,0.646952,0.068578,0.607351,0.076576,0.467249,0.125915
50%,0.653340,0.077129,0.629080,0.083284,0.504119,0.130377
75%,0.672640,0.082877,0.641087,0.088200,0.513975,0.139647
max,0.685562,0.091052,0.663263,0.092653,0.549410,0.146818
